**1. Prerequisites**

Enable GPU, if using google colab or AWS SageMaker Studio Lab.

**2. Find libraries**

By default, Google Colab is not able to run numba + CUDA, because two libraries are not found, libdevice and libnvvm.so . So we need to make sure that these libraries are found in the notebook.

In [1]:
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'

find: ‘/proc/31/task/31/net’: Invalid argument
find: ‘/proc/31/net’: Invalid argument
/usr/local/lib/python3.7/dist-packages/jaxlib/cuda/nvvm/libdevice
/usr/local/cuda-11.1/nvvm/libdevice
/usr/local/cuda-11.0/nvvm/libdevice
/usr/local/cuda-10.0/nvvm/libdevice
/usr/local/cuda-10.1/nvvm/libdevice
find: ‘/proc/31/task/31/net’: Invalid argument
find: ‘/proc/31/net’: Invalid argument
/usr/local/cuda-11.1/nvvm/lib64/libnvvm.so
/usr/local/cuda-11.0/nvvm/lib64/libnvvm.so
/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so
/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so


**3. Add two libraries to numba**

Add the two libraries to numba environment variables


In [2]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.0/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so"

In [3]:
import numpy as np
import math
from numba import cuda

In [4]:
cuda.detect()

Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

**Example for dealing with large input size**

From the example, we can compare the time by running multiple gpu threads parallel.

In [ ]:
@cuda.jit
def gpu_atan(x, out):
  idx = cuda.grid(1)
  out[idx] = math.atan(x[idx])
  
@cuda.jit
def gpu_atan_stride(x, out):
  start = cuda.grid(1)
  stride = cuda.gridsize(1)
  for i in range(start, x.shape[0], stride): 
    out[i] = math.atan(x[i])

In [ ]:
import numpy as np
a = np.arange(256*1000000,dtype=np.float32)
# move input data to the device
d_a = cuda.to_device(a)
# create output data on the device
d_out = cuda.device_array_like(d_a)

In [ ]:
%time gpu_atan_stride[1, 1](d_a, d_out); cuda.synchronize()


/usr/local/lib/python3.7/dist-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('https://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/usr/local/lib/python3.7/dist-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda-10.0/nvvm/libdevice.

For more information about alternatives visit: ('https://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))


CPU times: user 3min 47s, sys: 381 ms, total: 3min 48s
Wall time: 3min 47s


In [ ]:
# RUN THIS TWICE. 
# sometimes, the first time, the timing is way too large for some reason, 
# and not representative of the actual timing
# you should get something around 13 ms. 
%time gpu_atan[1000000,256](d_a, d_out); cuda.synchronize()

CPU times: user 153 ms, sys: 9.01 ms, total: 162 ms
Wall time: 161 ms


**Example : Calculate square of matrix and sum the result**

In [ ]:
@cuda.jit
def gpu_sqr(out): 
  # get the thread coordinates in 2D
  i1, i2 = cuda.grid(2)
  out[i1][i2] = out[i1][i2]*out[i1][i2]

@cuda.jit
def gpu_add(a, b, out):
  i1, i2 = cuda.grid(2)
  out[i1][i2] = a[i1][i2] + b[i1][i2]


In [ ]:
a = np.arange(12).reshape(3,4)
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [ ]:
d_a = cuda.to_device(a)
# we use two blocks, side-by-side in the horizontal direction
blocks = (1,2)
# each block has 6 threads arranged in 3 lines and 2 columns
threads_per_block = (3,2)
gpu_sqr[blocks, threads_per_block](d_a)
d_a.copy_to_host()

array([[  0,   1,   4,   9],
       [ 16,  25,  36,  49],
       [ 64,  81, 100, 121]])

In [ ]:
a = np.arange(12).reshape(3,4)
b = np.arange(0,120,10).reshape(3,4)
print(a)
print(b)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[  0  10  20  30]
 [ 40  50  60  70]
 [ 80  90 100 110]]


In [ ]:
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_out = cuda.device_array_like(d_a)
# we use two blocks, side-by-side in the horizontal direction
blocks = (1,2)
# each block has 6 threads arranged in 3 lines and 2 columns
threads_per_block = (3,2)
gpu_sqr[blocks, threads_per_block](d_a)
gpu_sqr[blocks, threads_per_block](d_b)

gpu_add[blocks, threads_per_block](d_a, d_b, d_out)

print(d_a.copy_to_host())
print(d_b.copy_to_host())
print(d_out.copy_to_host())

[[  0   1   4   9]
 [ 16  25  36  49]
 [ 64  81 100 121]]
[[    0   100   400   900]
 [ 1600  2500  3600  4900]
 [ 6400  8100 10000 12100]]
[[    0   101   404   909]
 [ 1616  2525  3636  4949]
 [ 6464  8181 10100 12221]]


**Reference**

1. [Cuda Kernel Python](https://thedatafrog.com/en/articles/cuda-kernel-python/)
2. [Numba Cuda Example](https://github.com/cbernet/maldives/blob/master/numba/numba_cuda.ipynb)
3. [Cuda Architecture](http://tdesell.cs.und.edu/lectures/cuda_2.pdf)